In [1]:
function deferred_acceptance(prop_prefs,resp_prefs,caps)

    m = size(prop_prefs,2)
    n = size(resp_prefs,2)
    prop_matches = zeros(Int64,m)
    L = sum(caps)
    resp_matches = zeros(Int64,L)
    unchanged_counter = 0
    next_m_approach = ones(Int64,m)
    
    indptr = Array(Int,n+1)
    indptr[1] = 1
    for i in 1:n
        indptr[i+1] = indptr[i] + caps[i]
    end
    
    while unchanged_counter < m
        unchanged_counter = 0
        for i in 1:m
            if prop_matches[i] == 0
                j = prop_prefs[next_m_approach[i],i]
                if j == 0
                    prop_matches[i] = 0
                    unchanged_counter += 1
                    
                else
                    a=resp_matches[indptr[j]:indptr[j+1]-1]
                    b=zeros(Int64,caps[j])
                    for k in 1:caps[j]
                        b[k] = findfirst(resp_prefs[:,j],a[k])
                    end
                    c = maximum(b)
                    if c > findfirst(resp_prefs[:,j],i)
                        prop_matches[i] = j
                        r = findfirst(a,resp_prefs[c,d])
                        if resp_matches[indptr[j]-1+r] != 0
                            prop_matches[resp_prefs[c,d]] = 0
                            next_m_approach[resp_prefs[c,d]] += 1
                        end
                        resp_matches[indptr[j]-1+r] = i
                    else #Žó‚¯“ü‚ê‚È‚¢‚Æ‚«
                        next_m_approach[i] += 1
                    end
                end
            else unchanged_counter += 1 
            end
        end
    end
    return prop_matches,resp_matches,indptr
end

function deferred_acceptance(prop_prefs,resp_prefs)
    caps = ones(Int, size(resp_prefs, 2))
    prop_matches, resp_matches, indptr =
        deferred_acceptance(prop_prefs, resp_prefs, caps)
    return prop_matches, resp_matches
end

deferred_acceptance (generic function with 2 methods)